# SLURM magics

## Introduction

text

text

## Using SLURM magics

``slurm-magic`` is available in the ``prace`` environment. To use it, type

In [ ]:
import slurm_magic
%load_ext slurm_magic

Check the magics provided by ``slurm-magic``

In [ ]:
%lsmagic

and try them out

In [ ]:
%squeue

In [ ]:
%sinfo

## GROMACS as an example

[GROMACS](http://www.gromacs.org/) is a molecular dynamics simulation package designed for simulations of biological macromolecules (proteins, lipids, nucleic acids, etc.).

In this exercise we use [lysozyme in water](http://www.mdtutorials.com/gmx/lysozyme/index.html) as a model system to demonstrate how to use Jupyter notebook to submit jobs and analyze results.

First, go to the ``gromacs_job`` folder

In [ ]:
cd gromacs_job

and check that the input files (.mdp, .top and .gro) are in the folder

In [ ]:
ls

Then, use the ``%%sbatch`` cell magic to submit a GROMACS job

In [ ]:
%%sbatch
#!/bin/bash -l
#SBATCH -A edu18.prace
#SBATCH -N 1
#SBATCH -t 1:00:00
#SBATCH -J gromacs
module load gromacs/2018.3
gmx_seq grompp -f npt.mdp -c start.gro -p topol.top
gmx_seq mdrun -s topol.tpr -deffnm npt

Monitor your job with the ``%squeue`` line magic

In [ ]:
%squeue -u <username>

As the simulation goes on, the output files will be constantly updated. You can start to analyze the output files and monitor the progress of the simulation.

We have prepared a Python module ``gmx_util`` that provides some easy-to-use functions for analysis. To use the module, type

In [ ]:
import gmx_util as gu

and read the documentation of the module via

In [ ]:
help(gu)

Now we are ready to do some analysis. Note that the GROMACS module is not loaded yet, and we need to do the following to add GROMACS executables to the $PATH environment variable

In [ ]:
gu.load_gmx()

Let's examine the evolution of density with respect to simulation time

In [ ]:
time,dens = gu.get_prop("Density","npt")

and plot density against time

In [ ]:
import matplotlib.pyplot as plt
plt.plot(time,dens)

Note that the default unit is kg/m<sup>3</sup> for density and ps for simulationt time. You may improve the plot by adding ``xlabel``, ``ylabel``, etc.

We can also examine the evolution of pressure with respect to time

In [ ]:
time,pres = gu.get_prop("Pressure","npt")
plt.plot(time,pres)

Also look at the correlation between density and pressure

In [ ]:
plt.plot(dens,pres[:len(dens)],'b+')

Since we are simulating lysozyme in water, we can monitor the root-mean-square deviation (RMSD) of protein

In [ ]:
time,rmsd = gu.get_rmsd("Backbone","npt")
plt.plot(time,rmsd)

In many cases it is of interest to analyze the coordinates of protein and surrounding solvent molecules. To do that, we first convert the binary trajectory file into the protein data bank (PDB) format

In [ ]:
gu.get_pdb("System","npt")

Then we extract a frame at e.g. 100 ps, and print some information about the atoms and residues

In [ ]:
atoms = gu.read_pdb(100, "npt")
print("Number of atoms:", len(atoms))
residues = list(set([a.resname for a in atoms]))
print("Number residue types:", len(residues))
print("Residue types:", residues)

Below is an example code for computing the shortest distance between protein and the first 30 water molecules

In [ ]:
import math

pro_atoms = []
wat_atoms = []
for a in atoms:
    if a.resname == "SOL":
        wat_atoms.append(a)
    elif a.resname == " CL":
        pass
    else:
        pro_atoms.append(a)

min_r2 = 1e+99
for a in pro_atoms:
    for b in wat_atoms[:90]:
        dx = a.x - b.x
        dy = a.y - b.y
        dz = a.z - b.z
        r2 = dx**2 + dy**2 + dz**2
        if min_r2 > r2:
            min_r2 = r2

print(math.sqrt(min_r2))

You can do other things with the coordinates, for instance computing the centroid of the protein

In [ ]:
# write your code to compute the centroid of the protein
